In [323]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

## Data Loading 

In [ ]:
df = pd.read_excel("Online Retail.xlsx")
print("Number of rows in the dataset: ", df.shape[0])
print("Number of columns in the dataset: ", df.shape[1])

In [ ]:
df.head()

## Data Inspection & Data Cleaning

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
cancelled = df[df['InvoiceNo'].astype(str).str.startswith('C')]
negative_price = df[df['UnitPrice'] < 0]
negative_quantity = df[df['Quantity'] < 0]
print("Number of cancelled orders: ", cancelled.shape[0])
print("Number of orders with price corrections: ", negative_price.shape[0])
print("Number of rows with negative 'Quantity': ", negative_quantity.shape[0])

The dataset contains negative values for both Quantity and UnitPrice. Negative values for Quantity likely represent canceled orders, which we can identify by checking if InvoiceNo starts with the letter 'C'. These transactions are separated into a new dataframe for deeper insights, such as analyzing patterns in cancellations, frequent reasons, or products that are often returned.

Negative values for UnitPrice might indicate corrections or anomalies in pricing, which could distort the analysis. These entries have been removed to ensure the integrity and accuracy of our findings, as retaining them could bias metrics like average sales or trends.

In [ ]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

Lastly, we check for the number of duplicated orders and remove any missing values. 

In [ ]:
print("Number of duplicated orders:", len(df[df.duplicated()]))

In [ ]:
df.drop_duplicates(inplace = True)
df = df.dropna()

In [ ]:
print("Number of rows in the dataset: ", df.shape[0])

In [ ]:
df.isnull().sum()

## Feature Engineering

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#### Calculating Total Sales

In [ ]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

#### Adding Hour, Day, Month, and Year Features 

In [ ]:
df['Hour'] = df['InvoiceDate'].dt.hour
df['Day'] = df['InvoiceDate'].dt.day_name()
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceYear']  = df['InvoiceDate'].dt.year
df.head(5)

## Product Analysis

### Monthly Sales Trend

In [ ]:
def most_purchased(month):
    monthly_data = df[df['InvoiceMonth'] == month]
    top_product = (monthly_data.groupby('Description')['Quantity']
                   .sum().sort_values(ascending=False).idxmax())
    return top_product

monthly_sales = df.groupby('InvoiceMonth')['TotalSales'].sum().reset_index()
monthly_sales['MonthName'] = monthly_sales['InvoiceMonth'].apply(lambda x: calendar.month_name[x])
monthly_sales['MostPurchasedProduct'] = monthly_sales['InvoiceMonth'].apply(most_purchased)
monthly_sales

In [ ]:
sns.set_theme(style="whitegrid", context="talk")
custom_palette = ["#2A9D8F", "#E9C46A", "#F4A261"]

In [ ]:
def format_y_axis(value, tick_number):
    return f'{int(value):,}'

fig, ax = plt.subplots(figsize=(12, 6))

ax.bar(
    monthly_sales['MonthName'],
    monthly_sales['TotalSales'],
    color=custom_palette[0],  
    edgecolor='black',
    width=0.8
)

ax.set_title('Monthly Sales Trend', fontsize=16, pad=15)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Sales (in USD)', fontsize=12)
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

ax.yaxis.set_major_formatter(FuncFormatter(format_y_axis))
ax.grid(False)

ax.grid(axis='y', linestyle='--', alpha=0.7)

for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor('lightgrey')
    spine.set_linewidth(1.5)

plt.tight_layout()

plt.show()

### Top 10 Products in 2010

In [ ]:
top_2010_products = df[df['InvoiceYear'] == 2010]
top_products_2010 = (
    top_2010_products.groupby('Description')['Quantity'].sum()
    .nlargest(10)
    .reset_index()
)

top_2010_months = []
for product in top_products_2010['Description']:
    product_data = top_2010_products[top_2010_products['Description'] == product]
    top_month = (
        product_data.groupby('InvoiceMonth')['Quantity'].sum()
        .idxmax()
    )
    top_2010_months.append(calendar.month_name[top_month])

top_products_2010['MostPurchasedMonth'] = top_2010_months
top_products_2010

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.bar(
    top_products_2010['Description'],
    top_products_2010['Quantity'],
    color=custom_palette[2],  
    edgecolor='black',
    width=0.8
)

ax.set_title('Top 10 Products by Quantity Sold in 2010', fontsize=16, pad=15)
ax.set_xlabel('Product Description', fontsize=12)
ax.set_ylabel('Quantity Sold', fontsize=12)
ax.tick_params(axis='x', labelrotation=45, labelsize=10, direction='out')
ax.tick_params(axis='y', labelsize=10)

ax.grid(False)

for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor('lightgrey')
    spine.set_linewidth(1.5)

plt.tight_layout()

plt.show()

### Top 10 Products in 2011

In [ ]:
top_2011_products = df[df['InvoiceYear'] == 2011]
top_products_2011 = (
    top_2011_products.groupby('Description')['Quantity'].sum()
    .nlargest(10)
    .reset_index()
)

top_2011_months = []
for product in top_products_2011['Description']:
    product_data = top_2011_products[top_2011_products['Description'] == product]
    top_month = (
        product_data.groupby('InvoiceMonth')['Quantity'].sum()
        .idxmax()
    )
    top_2011_months.append(calendar.month_name[top_month])

top_products_2011['MostPurchasedMonth'] = top_2011_months
top_products_2011

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.bar(
    top_products_2011['Description'],
    top_products_2011['Quantity'],
    color=custom_palette[1],  
    edgecolor='black',
    width=0.8
)

ax.set_title('Top 10 Products by Quantity Sold in 2011', fontsize=16, pad=15)
ax.set_xlabel('Product Description', fontsize=12)
ax.set_ylabel('Quantity Sold', fontsize=12)
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

ax.grid(False)

plt.tight_layout()

plt.show()

## Customer Segmentation with K-means Clustering

### RFM (Recency, Frequency, Monetary) Metrics 

In [ ]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (pd.Timestamp.now() - x.max()).days,  
    'InvoiceNo': 'count',                                         
    'TotalSales': 'sum'                                          
}).rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSales': 'Monetary'
})

### Data Normalization

In [ ]:
scaler = StandardScaler()
rfm_normalized = scaler.fit_transform(rfm)

### Calculating the optimal number of clusters

In [ ]:
inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_normalized)
    inertia.append(kmeans.inertia_)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(K, inertia, marker='x', linestyle='-', color=custom_palette[0], linewidth=2, markersize=8)
ax.set_title('Elbow Plot for Optimal K', fontsize=16, pad=15)
ax.set_xlabel('Number of Clusters', fontsize=12)
ax.set_ylabel('Inertia', fontsize=12)

ax.grid(False)

plt.show()

Based on the plot above, the optimal number of clusters is K = 4, where the reduction in inertia slows down noticeably after that point.

In [ ]:
optimal_k = 4 
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_normalized)

### Cluster Analysis

In [ ]:
var = ['Monetary', 'Frequency', 'Recency']
fig, ax = plt.subplots(figsize=(15, 5))

centroids.plot(
    kind='bar',
    ax=ax,
    color=custom_palette,
    width=0.8
)

ax.grid(False)

ax.set_title('Cluster Centroids', fontsize=16, pad=15)
ax.set_ylabel('Centroid Value', fontsize=12)
ax.set_xlabel('Cluster', fontsize=12)

ax.set_xticklabels(centroids.index, rotation=0)

for spine in ax.spines.values():
    spine.set_visible(True)  
    spine.set_edgecolor('lightgrey')  
    spine.set_linewidth(1.5)  

legend = ax.legend(
    title='Metrics',
    loc='upper left',
    fontsize=10,
    title_fontsize=12
)

plt.tight_layout()

plt.show()

In [ ]:
print(rfm['Cluster'].value_counts())

### Key Observations of Customer Segmentation

**Cluster 0**: 
 - Typically below average per‐customer spending, since the cluster centroid for Monetary is below zero.
 - Frequency and Recency is around the overall mean.
 - These customers spend a bit less than average per purchase but buy with about average frequency and recency. In raw (non‐standardized) terms, Cluster 0’s totals may appear large simply because it contains the most customers, but at an individual level, spending is somewhat modest.

**Cluster 1**: 
 - Monetary value is above average compared to other clusters .
 - Frequency and recency is lower than average.
 - This segment tends to spend more per transaction than the mean, but they do not buy as frequently or as recently as other groups. They may be higher‐ticket shoppers who purchase infrequently.

**Cluster 2**: 
 - Has a very high centroid for Frequency and moderate recency.
 - Monetary value is below average.
 - On a per‐customer basis, these buyers place many orders (high Frequency), but each order is typically for a lower amount (below‐average Monetary). In the raw charts, this cluster has relatively small totals (e.g., total Monetary) because the group itself is small (only 7 customers). Individually, though, they are very active purchasers.
 
**Cluster 3**: 
 - A moderate or modest Monetary value relative to the mean.
 - A lower Frequency centroid compared to Cluster 2 but higher than average compared to Clusters 0 and 1.
 - A high Recency, suggesting these are very recent purchasers on average.
 - These customers buy fairly often and very recently, with spending levels around the overall mean. Though their total Recency contribution in raw terms is not huge (due to smaller cluster size), their per‐customer recency is among the highest.

### Suggestions for Targeted Strategies